In [15]:
import os
import glob
import re
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from scipy.ndimage import uniform_filter1d # For smoothing


colors = {
	'AW': '#ef8a62',
	'MO': '#67a9cf',
	'MI': '#999999',
}

# === CONFIG ===
data_dir = 'ERP_exports/'  # Updated to match MATLAB output_folder
channels = ['Fz', 'C3', 'Cz', 'C4', 'Pz', 'Oz'] # Updated channel list
conditions = ['AW', 'MI', 'MO']
tasks = ['Arm', 'Leg']
smoothing_window = 15  # Smoothing window in samples

# === LOAD AND ORGANIZE ===
erp_data = {ch: {task: {cond: [] for cond in conditions} for task in tasks} for ch in channels}
time_vector = None
expected_len_global = None

all_files = glob.glob(os.path.join(data_dir, '*.csv'))
if not all_files:
	print(f"No CSV files found in directory: {os.path.abspath(data_dir)}")
else:
	# Determine expected length and time_vector from the first valid file
	for file_path in all_files:
		try:
			temp_df = pd.read_csv(file_path)
			if not temp_df.empty and 'Time' in temp_df.columns and 'ERP' in temp_df.columns and temp_df['ERP'].values.size > 0:
				expected_len_global = len(temp_df['ERP'].values)
				time_vector = temp_df['Time'].values
				print(f"Determined expected data length: {expected_len_global} and time_vector from {file_path}")
				break
		except Exception as e:
			print(f"Minor error reading initial file {file_path} for metadata: {e}")
			continue # Try next file

	if expected_len_global is None or time_vector is None:
		print("Error: Could not determine a valid data length/time_vector from any CSV file. Aborting plotting script.")
		exit() # Exit if essential metadata couldn't be loaded
	
	# Regex for new filename format: e.g., 1_C3_AW_Arm.csv
	# Allows for one or more digits for subjectID, and specific channel names
	filename_regex = re.compile(
		r'(?P<subjectID>\d+)_(?P<ch>(?:' + '|'.join(channels) + r'))_(?P<cond>AW|MI|MO)_(?P<task>Arm|Leg)\.csv'
	)

	for file_path in all_files:
		filename = os.path.basename(file_path)
		match = filename_regex.match(filename)
		
		if match:
			# subject_id_str = match.group('subjectID') # Not directly used for dict keys in this script
			ch_name = match.group('ch')
			cond_name = match.group('cond')
			task_name = match.group('task')
			
			# Ensure ch_name, cond_name, task_name are valid as per config (already in regex)
			if ch_name not in channels or cond_name not in conditions or task_name not in tasks:
				print(f"Warning: Parsed components from {filename} not in configured lists. Skipping.")
				continue

			try:
				df_csv = pd.read_csv(file_path)
				if 'ERP' in df_csv.columns and len(df_csv['ERP'].values) == expected_len_global:
					erp_waveform = df_csv['ERP'].values
					erp_data[ch_name][task_name][cond_name].append(erp_waveform)
				elif 'ERP' in df_csv.columns: # Has ERP but wrong length
					print(f"Warning: File {filename} ERP data length {len(df_csv['ERP'].values)} != expected {expected_len_global}. Skipping.")
				else:
					print(f"Warning: File {filename} missing 'ERP' column. Skipping.")
			except Exception as e:
				print(f"Warning: Could not process file {filename}. Error: {e}. Skipping.")
		# else:
		#     print(f"Filename {filename} did not match expected pattern. Skipping.")


# === PLOT ERPs ===
for ch in channels:
    for task in tasks:
        fig = go.Figure()
        
        # PLOT ERP + SEM
        has_data_for_plot = False
        for cond in conditions:
            trials_list = erp_data[ch][task][cond]
            if not trials_list:
                continue
            try:
                trials = np.array(trials_list)
                if trials.ndim == 1 and trials.size > 0:
                    trials = trials.reshape(1, -1)
                elif trials.size == 0:
                    continue
            except Exception as e:
                print(f"Error converting trials to array for {ch}-{task}-{cond}: {e}. Skipping.")
                continue

            if trials.shape[0] == 0 or trials.shape[1] != expected_len_global:
                print(f"Data shape issue for {ch}-{task}-{cond}: {trials.shape} vs expected ({trials.shape[0]},{expected_len_global}). Skipping.")
                continue
            
            n_subjects = trials.shape[0]
            if n_subjects == 0: continue

            has_data_for_plot = True
            smoothed = uniform_filter1d(trials, size=smoothing_window, axis=1)
            mean_erp = smoothed.mean(axis=0)
            sem = smoothed.std(axis=0) / np.sqrt(n_subjects)
            
            fig.add_trace(go.Scatter(
                x=time_vector,
                y=mean_erp,
                mode='lines',
                name=cond,
                line=dict(color=colors[cond], width=2)
            ))

        # Add vertical dotted gray line at 0s
        fig.add_shape(
            type="line",
            x0=0, x1=0,
            y0=min(mean_erp - sem), y1=max(mean_erp + sem),
            line=dict(color="gray", width=2, dash="dot"),
            layer="below"
        )
        
        if has_data_for_plot:
            fig.update_layout(
                title=f"ERP at {ch} - Task: {task}",
                xaxis_title="Time (ms)",
                yaxis_title="Amplitude (µV)",
                template="simple_white",
                legend_title="Condition",
                width=950,
                height=450, 
                xaxis=dict(range=[-500, 3000], tickmode='linear', dtick=500),
				showlegend=False
            )
            fig.show()
        else:
            print(f"No data to plot for {ch} - {task} after filtering.")

Determined expected data length: 1000 and time_vector from ERP_exports\10_C3_AW_Arm.csv


In [16]:
# Store ERP data for each channel and task as a row, including the time window
erp_summary_rows = []
for ch in channels:
    for task in tasks:
        # Collect all trials for all conditions for this channel and task
        for cond in conditions:
            trials_list = erp_data[ch][task][cond]
            if trials_list:
                erp_summary_rows.append({
                    'channel': ch,
                    'task': task,
                    'condition': cond,
                    'time_window': (time_vector[0], time_vector[-1]),
                    'ERP': trials_list  # list of numpy arrays (one per subject)
                })
erp_summary_df = pd.DataFrame(erp_summary_rows)
erp_summary_df

,channel,task,condition,time_window,ERP
0,Fz,Arm,AW,"(-1000, 2996)","[[1.261382, 2.998709, 5.173142, 6.459421, 5.97..."
1,Fz,Arm,MI,"(-1000, 2996)","[[3.652727, 1.692406, 0.7518276, 0.775896, 1.4..."
2,Fz,Arm,MO,"(-1000, 2996)","[[-1.173524, -0.9484189, -0.4536947, 0.1411034..."
3,Fz,Leg,AW,"(-1000, 2996)","[[3.911951, 3.85537, 3.395594, 2.820391, 2.122..."
4,Fz,Leg,MI,"(-1000, 2996)","[[0.1321689, 0.02350035, 0.162132, 0.05950511,..."
5,Fz,Leg,MO,"(-1000, 2996)","[[3.906321, 5.109617, 6.452438, 7.207507, 6.98..."
6,C3,Arm,AW,"(-1000, 2996)","[[4.013222, 6.217203, 8.93599, 10.65785, 10.32..."
7,C3,Arm,MI,"(-1000, 2996)","[[1.873623, 0.3323835, -0.7678012, -1.51116, -..."
8,C3,Arm,MO,"(-1000, 2996)","[[-6.91469, -7.035739, -7.001485, -6.926352, -..."
9,C3,Leg,AW,"(-1000, 2996)","[[12.41797, 12.12515, 11.72418, 11.42187, 11.0..."


In [17]:
import os
import glob
import re
import numpy as np
import pandas as pd
from scipy.stats import f_oneway, ttest_ind
from statsmodels.stats.multitest import fdrcorrection

# === CONFIG ===
data_dir = 'ERP_exports/'  # Folder with all CSV files
channels = ['C3', 'Cz', 'C4', 'Fz', 'Pz', 'Oz']
conditions = ['AW', 'MI', 'MO']
tasks = ['Arm', 'Leg']
alpha = 0.05  # Significance level

# === LOAD AND ORGANIZE DATA ===
erp_data = {ch: {task: {cond: [] for cond in conditions} for task in tasks} for ch in channels}
time_vector = None
expected_len_global = None

all_files = glob.glob(os.path.join(data_dir, '*.csv'))
if not all_files:
    print(f"No CSV files found in directory: {data_dir}")
else:
    for file_path in all_files:  # Find first valid file with Time and ERP columns
        try:
            df = pd.read_csv(file_path)
            if {'Time', 'ERP'}.issubset(df.columns) and not df.empty:
                expected_len_global = len(df['ERP'])
                time_vector = df['Time'].values
                print(f"Determined expected data length: {expected_len_global} points from {file_path}")
                break
        except Exception:
            continue

    if expected_len_global is None:
        print("Error: Could not determine a valid data length from any CSV file. Aborting stats.")
    else:
        for file_path in all_files:
            filename = os.path.basename(file_path)
            match = re.match(r'\d+_(?P<ch>C3|Cz|C4|Fz|Oz|Pz)_(?P<cond>AW|MI|MO)_(?P<task>Arm|Leg)\.csv', filename)
            if match:
                ch = match.group('ch')
                cond_name_from_file = match.group('cond')
                task = match.group('task')
                try:
                    df = pd.read_csv(file_path)
                    if {'Time', 'ERP'}.issubset(df.columns):
                        data = df['ERP'].values
                        if len(data) == expected_len_global:
                            erp_data[ch][task][cond_name_from_file].append(data)
                        elif len(data) > 0:
                            print(f"Warning: File {filename} data length {len(data)} != expected {expected_len_global}. Skipping file.")
                except Exception:
                    pass

# === STATISTICAL ANALYSIS AND TABULAR OUTPUT ===
significance_summary_list = []

if time_vector is None:
    print("Time vector not initialized. Cannot perform statistical analysis.")
else:
    for ch in channels:
        for task in tasks:
            condition_data_for_anova = {}

            for cond_name in conditions:
                trials = np.array(erp_data[ch][task][cond_name])
                if trials.ndim == 1 and trials.size == 0:
                    trials = np.empty((0, len(time_vector)))
                elif trials.ndim == 1 and trials.size > 0:
                    trials = trials.reshape(1, -1)

                if trials.shape[0] > 0 and trials.shape[1] == len(time_vector):
                    condition_data_for_anova[cond_name] = trials

            if not all(c in condition_data_for_anova for c in conditions) or len(condition_data_for_anova) < 2:
                continue

            anova_input_arrays = [condition_data_for_anova[c] for c in conditions if c in condition_data_for_anova]
            if len(anova_input_arrays) < 2:
                continue

            num_time_points = anova_input_arrays[0].shape[1]
            pvals_anova = []
            for t in range(num_time_points):
                samples_at_t = [arr[:, t] for arr in anova_input_arrays]
                samples_at_t_filtered = [s for s in samples_at_t if len(s) >= 2]
                if len(samples_at_t_filtered) < 2:
                    pvals_anova.append(1.0)
                    continue
                try:
                    _, p = f_oneway(*samples_at_t_filtered)
                    pvals_anova.append(p)
                except ValueError:
                    pvals_anova.append(1.0)

            if not pvals_anova:
                continue

            pvals_anova = np.array(pvals_anova)
            _, pvals_fdr_anova = fdrcorrection(pvals_anova, alpha=alpha, method='indep')
            sig_mask_anova = pvals_fdr_anova < alpha

            in_region = False
            region_start_idx = 0
            for i in range(num_time_points):
                if sig_mask_anova[i] and not in_region:
                    region_start_idx = i
                    in_region = True

                if ((not sig_mask_anova[i]) or (i == num_time_points - 1)) and in_region:
                    actual_end_idx_of_block = i if sig_mask_anova[i] else i - 1
                    if actual_end_idx_of_block < region_start_idx:
                        in_region = False
                        continue

                    start_time = time_vector[region_start_idx]
                    end_time = time_vector[actual_end_idx_of_block]
                    min_p_anova_interval = np.min(pvals_fdr_anova[region_start_idx:actual_end_idx_of_block + 1])

                    current_summary_entry = {
                        'Channel': ch,
                        'Task': task,
                        'Start Time (ms)': round(start_time, 2),
                        'End Time (ms)': round(end_time, 2),
                        'Duration (ms)': round(end_time - start_time, 2),
                        'Min ANOVA FDR p-value': round(min_p_anova_interval, 5)
                    }

                    cond_pairs_to_test = [("AW", "MI"), ("AW", "MO"), ("MI", "MO")]
                    interval_post_hoc_stats = {
                        f"{p[0]}_vs_{p[1]}": {"sig_count": 0, "dir1_gt_dir2_count": 0, "dir2_gt_dir1_count": 0}
                        for p in cond_pairs_to_test
                    }

                    for t in range(region_start_idx, actual_end_idx_of_block + 1):
                        p_values_for_pairs_at_t = []
                        mean_values_at_t = {}
                        testable_pairs_info_at_t = []

                        for c1_name, c2_name in cond_pairs_to_test:
                            if c1_name in condition_data_for_anova and c2_name in condition_data_for_anova:
                                data1_at_t = condition_data_for_anova[c1_name][:, t]
                                data2_at_t = condition_data_for_anova[c2_name][:, t]

                                if c1_name not in mean_values_at_t:
                                    mean_values_at_t[c1_name] = np.mean(data1_at_t)
                                if c2_name not in mean_values_at_t:
                                    mean_values_at_t[c2_name] = np.mean(data2_at_t)

                                if len(data1_at_t) >= 2 and len(data2_at_t) >= 2:
                                    _, p_val = ttest_ind(data1_at_t, data2_at_t, equal_var=False)
                                    p_values_for_pairs_at_t.append(p_val)
                                    testable_pairs_info_at_t.append((f"{c1_name}_vs_{c2_name}", c1_name, c2_name))
                                else:
                                    p_values_for_pairs_at_t.append(np.nan)
                                    testable_pairs_info_at_t.append((f"{c1_name}_vs_{c2_name}", None, None))
                            else:
                                p_values_for_pairs_at_t.append("n.t.")
                                testable_pairs_info_at_t.append((f"{c1_name}_vs_{c2_name}", None, None))

                        actual_p_values_to_correct = [p for p in p_values_for_pairs_at_t if isinstance(p, float) and not np.isnan(p)]
                        indices_of_actual_p_values = [i for i, p in enumerate(p_values_for_pairs_at_t) if isinstance(p, float) and not np.isnan(p)]

                        if actual_p_values_to_correct:
                            rejected_pairwise, _ = fdrcorrection(actual_p_values_to_correct, alpha=alpha, method='indep')
                            for k, original_idx in enumerate(indices_of_actual_p_values):
                                pair_key, c1_n, c2_n = testable_pairs_info_at_t[original_idx]
                                if rejected_pairwise[k]:
                                    interval_post_hoc_stats[pair_key]["sig_count"] += 1
                                    mean_c1 = mean_values_at_t.get(c1_n, np.nan)
                                    mean_c2 = mean_values_at_t.get(c2_n, np.nan)
                                    if not np.isnan(mean_c1) and not np.isnan(mean_c2):
                                        if mean_c1 > mean_c2:
                                            interval_post_hoc_stats[pair_key]["dir1_gt_dir2_count"] += 1
                                        elif mean_c2 > mean_c1:
                                            interval_post_hoc_stats[pair_key]["dir2_gt_dir1_count"] += 1

                    num_pts_in_interval = actual_end_idx_of_block - region_start_idx + 1
                    for c1_n_ph, c2_n_ph in cond_pairs_to_test:
                        pair_key_ph = f"{c1_n_ph}_vs_{c2_n_ph}"
                        stats = interval_post_hoc_stats[pair_key_ph]
                        desc = "n.s."

                        if not (c1_n_ph in condition_data_for_anova and c2_n_ph in condition_data_for_anova):
                            desc = "n.t."
                        elif stats["sig_count"] > 0:
                            dir1_count = stats["dir1_gt_dir2_count"]
                            dir2_count = stats["dir2_gt_dir1_count"]
                            direction_str = ""
                            if dir1_count > dir2_count and dir1_count >= 0.5 * stats["sig_count"]:
                                direction_str = f"{c1_n_ph} > {c2_n_ph}"
                            elif dir2_count > dir1_count and dir2_count >= 0.5 * stats["sig_count"]:
                                direction_str = f"{c2_n_ph} > {c1_n_ph}"
                            elif dir1_count > 0 or dir2_count > 0:
                                direction_str = "Mixed"

                            desc = f"Sig ({stats['sig_count']}/{num_pts_in_interval} pts"
                            if direction_str:
                                desc += f"; {direction_str}"
                            desc += ")"
                        current_summary_entry[f'Pairwise_{pair_key_ph}'] = desc

                    significance_summary_list.append(current_summary_entry)
                    in_region = False

# === OUTPUT SUMMARY ===
if significance_summary_list:
    summary_df = pd.DataFrame(significance_summary_list)
    cols_order = ['Channel', 'Task', 'Start Time (ms)', 'End Time (ms)', 'Duration (ms)', 'Min ANOVA FDR p-value']
    for c1, c2 in [('AW', 'MI'), ('AW', 'MO'), ('MI', 'MO')]:
        cols_order.append(f'Pairwise_{c1}_vs_{c2}')
    for col in cols_order:
        if col not in summary_df.columns:
            summary_df[col] = 'n.a.'

    print("\n--- Significance Summary Table (with Pairwise Post-Hoc Info) ---")
    print(summary_df[cols_order].to_string())
    # summary_df.to_csv("erp_significance_summary_with_posthoc.csv", index=False)
else:
    print("\nNo significant regions found by overall ANOVA across all channels and tasks.")


Determined expected data length: 1000 points from ERP_exports\10_C3_AW_Arm.csv

--- Significance Summary Table (with Pairwise Post-Hoc Info) ---
   Channel Task  Start Time (ms)  End Time (ms)  Duration (ms)  Min ANOVA FDR p-value       Pairwise_AW_vs_MI         Pairwise_AW_vs_MO         Pairwise_MI_vs_MO
0       C3  Arm              296            308             12                0.02725                    n.s.    Sig (4/4 pts; AW > MO)    Sig (4/4 pts; MI > MO)
1       C3  Arm              324            348             24                0.00233                    n.s.    Sig (7/7 pts; AW > MO)    Sig (7/7 pts; MI > MO)
2       C3  Arm              360            360              0                0.03472                    n.s.    Sig (1/1 pts; AW > MO)    Sig (1/1 pts; MI > MO)
3       C3  Arm              392            408             16                0.02725                    n.s.    Sig (5/5 pts; AW > MO)    Sig (5/5 pts; MI > MO)
4       C3  Arm              428            4

In [18]:
import os
import glob
import re
import numpy as np
import pandas as pd
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import fdrcorrection

# === CONFIG ===
data_dir = 'ERP_exports/'  # Path to your CSV files
channels = ['C3', 'Cz', 'C4', 'Fz', 'Pz', 'Oz']
tasks = ['Arm', 'Leg']
comparisons = [('MI', 'MO'), ('AW', 'MI'), ('AW', 'MO')]
alpha = 0.05  # Significance threshold

# === LOAD DATA ===
erp_data = {ch: {task: {cond: {} for cond in ['AW', 'MI', 'MO']} for task in tasks} for ch in channels}
time_vector = None

all_files = glob.glob(os.path.join(data_dir, '*.csv'))

for file_path in all_files:
    filename = os.path.basename(file_path)
    match = re.match(r'(?P<subj>\d+?)_(?P<ch>C3|Cz|C4|Fz|Pz|Oz)_(?P<cond>AW|MI|MO)_(?P<task>Arm|Leg)\.csv', filename)
    if not match:
        continue

    subj = match.group('subj')
    ch = match.group('ch')
    cond = match.group('cond')
    task = match.group('task')

    df = pd.read_csv(file_path)
    if 'Time' not in df.columns or 'ERP' not in df.columns:
        continue

    if time_vector is None:
        time_vector = df['Time'].values

    erp_data[ch][task][cond][subj] = df['ERP'].values

# === ANALYSIS ===
summary = []

for ch in channels:
    for task in tasks:
        for cond1, cond2 in comparisons:
            subj_set = set(erp_data[ch][task][cond1].keys()) & set(erp_data[ch][task][cond2].keys())
            if len(subj_set) < 2:
                continue

            subj_list = sorted(list(subj_set))
            data1 = np.array([erp_data[ch][task][cond1][s] for s in subj_list])
            data2 = np.array([erp_data[ch][task][cond2][s] for s in subj_list])

            if data1.shape != data2.shape:
                continue

            pvals = []
            tstats = []
            for t in range(data1.shape[1]):
                t_stat, p_val = ttest_rel(data1[:, t], data2[:, t])
                pvals.append(p_val)
                tstats.append(t_stat)

            pvals = np.array(pvals)
            tstats = np.array(tstats)
            _, pvals_fdr = fdrcorrection(pvals, alpha=alpha)
            sig_mask = pvals_fdr < alpha

            in_sig_region = False
            region_start = 0

            for i in range(len(sig_mask)):
                if sig_mask[i] and not in_sig_region:
                    region_start = i
                    in_sig_region = True
                elif (not sig_mask[i] or i == len(sig_mask) - 1) and in_sig_region:
                    region_end = i if not sig_mask[i] else i
                    start_time = time_vector[region_start]
                    end_time = time_vector[region_end]
                    duration = end_time - start_time
                    min_p = np.min(pvals_fdr[region_start:region_end + 1])
                    mean_diff = np.mean(data1[:, region_start:region_end + 1] - data2[:, region_start:region_end + 1])
                    direction = f"{cond1} > {cond2}" if mean_diff > 0 else f"{cond2} > {cond1}"

                    summary.append({
                        'Channel': ch,
                        'Task': task,
                        'Comparison': f'{cond1} vs {cond2}',
                        'Start Time (ms)': round(start_time, 2),
                        'End Time (ms)': round(end_time, 2),
                        'Duration (ms)': round(duration, 2),
                        'Min FDR p-value': round(min_p, 5),
                        'Direction': direction
                    })
                    in_sig_region = False

# === OUTPUT ===
if summary:
    summary_df = pd.DataFrame(summary)
    print("\n--- Paired t-test Results with FDR Correction ---")
    print(summary_df.to_string(index=False))
    # summary_df.to_csv("paired_ttest_summary.csv", index=False)
else:
    print("No significant time regions found.")



--- Paired t-test Results with FDR Correction ---
Channel Task Comparison  Start Time (ms)  End Time (ms)  Duration (ms)  Min FDR p-value Direction
     C3  Arm   AW vs MO              324            348             24          0.00681   AW > MO
     C3  Arm   AW vs MO              360            364              4          0.02772   AW > MO
     Fz  Arm   MI vs MO              -68            -64              4          0.02217   MO > MI
     Fz  Arm   MI vs MO              -56            -52              4          0.03312   MO > MI
     Fz  Arm   MI vs MO              -36            -32              4          0.03434   MO > MI
     Fz  Arm   MI vs MO              -24            -20              4          0.03434   MO > MI
     Fz  Arm   MI vs MO               -4             20             24          0.01399   MO > MI
     Fz  Arm   MI vs MO               32             40              8          0.02217   MO > MI
     Fz  Arm   MI vs MO               48             56            